In [8]:
import os
from multiprocessing import Pool

import numpy as np
import pandas as pd

from DataImporter import achar_feathers, achar_ressonancias, agrupar_por_desgaste

In [9]:
TEST_PATH = "/home/gabrielmc/Documentos/LINSE/Dados"

In [10]:
all_feathers = achar_feathers(TEST_PATH)

In [11]:
sorted_feathers = agrupar_por_desgaste(all_feathers)

In [12]:
dfs = []
for desgaste_lvl, feathers in enumerate(sorted_feathers):
    if feathers == [None]:
        continue
    with Pool() as pool:
        inputs = [(feather, "imu1accz") for feather in feathers]
        ressonancias = list(pool.imap_unordered(achar_ressonancias, inputs))
        index = pd.MultiIndex.from_product([[desgaste_lvl], range(len(ressonancias))],
                                           names= ["Desgaste", "Amostra"])
        dfs.append(pd.DataFrame(ressonancias, index=index))

In [13]:
df_ressonancias = pd.concat(dfs)
df_ressonancias

0       1        2
Desgaste Amostra                         
0        0        29.952  81.952  161.200
         1        29.744  81.536  160.576
         2        29.952  81.536  160.784
         3        29.952  81.952  160.992
         4        29.952  81.744  160.992
...                  ...     ...      ...
7        19       29.952  81.536  156.832
         20       29.952  81.536  156.832
         21       29.952  81.536  156.832
         22       29.952  81.328  156.624
         23       29.744  81.328  156.624

[192 rows x 3 columns]

In [14]:
df_ressonancias.describe()

,0,1,2
count,192.000000,192.000000,192.000000
mean,29.826333,81.851250,160.330083
std,0.122192,0.238895,1.415423
min,29.536000,81.328000,156.624000
25%,29.744000,81.744000,160.316000
50%,29.744000,81.952000,160.992000
75%,29.952000,81.952000,160.992000
max,30.160000,82.160000,161.408000
